In [3]:
import openai
import os

In [2]:
from openai import OpenAI

In [5]:
client=OpenAI()

In [6]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":"is it too late to join the course?"}]
)

In [7]:
response

ChatCompletion(id='chatcmpl-9bgShXrkgxyjElJ9MyXaNiMt7zQCw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The answer to whether it's too late to join a course depends on several factors, such as the specific course you are considering, the enrollment deadlines set by the institution, and the progress already made in the course. Here are a few steps you can take to find out if you can still join:\n\n1. **Check Enrollment Deadlines**: Look for information on the course or program's official website. They often provide deadlines for registration and enrollment.\n\n2. **Contact the Institution**: Reach out to the admissions office or the course instructor directly. They can offer you the most definitive answer and may explain any late enrollment policies if they exist.\n\n3. **Online Platforms**: If the course is offered through an online learning platform (e.g., Coursera, edX, Udemy), check the course page. Online courses may have mor

In [11]:
response.choices[0].message.content

"The answer to whether it's too late to join a course depends on several factors, such as the specific course you are considering, the enrollment deadlines set by the institution, and the progress already made in the course. Here are a few steps you can take to find out if you can still join:\n\n1. **Check Enrollment Deadlines**: Look for information on the course or program's official website. They often provide deadlines for registration and enrollment.\n\n2. **Contact the Institution**: Reach out to the admissions office or the course instructor directly. They can offer you the most definitive answer and may explain any late enrollment policies if they exist.\n\n3. **Online Platforms**: If the course is offered through an online learning platform (e.g., Coursera, edX, Udemy), check the course page. Online courses may have more flexible start dates.\n\n4. **Catch-Up Policies**: Some courses may allow late enrollment if you can catch up on missed lectures and assignments. Inquire abou